In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

file = '데이터_날짜별 의정활동.xlsx'
df = pd.read_excel(file)

df.head()

,대수,일자,의안구분,단계,의안번호,의안명,세부단계,소관위원회,활동상태
0,02,0,건의안,접수,020620,농업요원확보에관한건의안,제안,-,발의
1,02,0,건의안,접수,020631,학도의징집과군사훈련방법에관한건의안,제안,-,발의
2,03,0,건의안,접수,030574,미곡수출에관한건의안,제안,-,발의
3,03,0,건의안,접수,030580,영농자금대출에관한건의안,제안,-,발의
4,03,0,건의안,접수,030609,판자집철거대책에관한건의안,제안,-,발의


# 결측치 확인

In [8]:
print('전체 데이터 건수:', len(df))

전체 데이터 건수: 874835


In [19]:
print('컬럼별 결측치 개수')
df.isnull().sum().to_frame('nan_count')

컬럼별 결측치 개수


,nan_count
대수,0
일자,0
의안구분,0
단계,179
의안번호,0
의안명,0
세부단계,0
소관위원회,2
활동상태,0


In [20]:
# 결측치 비율 파악
pd.DataFrame(data=df.isnull().sum()/len(df), columns=['nan_ratio'])

,nan_ratio
대수,0.000000
일자,0.000000
의안구분,0.000000
단계,0.000205
의안번호,0.000000
의안명,0.000000
세부단계,0.000000
소관위원회,0.000002
활동상태,0.000000


In [35]:
# 결측치 제거
df.dropna(inplace=True)
pd.DataFrame(data=df.isnull().sum()/len(df), columns=['nan_ratio'])

,nan_ratio
일자,0.0
의안구분,0.0
단계,0.0
의안번호,0.0
의안명,0.0
세부단계,0.0
소관위원회,0.0
활동상태,0.0


# Column 값 확인

In [38]:
date_types = df['일자'].unique
print("일자: {}".format(date_types))
law_types = df['의안구분'].unique()
print("의안구분: {}".format(law_types))
turn_types = df['단계'].unique()
print("단계: {}".format(turn_types))
lawNum_types = df['의안번호'].unique()
print("의안번호: {}".format(lawNum_types))
lawName_types = df['의안명'].unique()
print("의안명: {}".format(law_types))
specific_types = df['세부단계'].unique()
print("세부단계: {}".format(specific_types))
insti_types = df['소관위원회'].unique()
print("소관위원회: {}".format(insti_types))
active_types = df['활동상태'].unique()
print("활동상태: {}".format(active_types))

일자: <bound method Series.unique of 0                  0
1                  0
2                  0
3                  0
4                  0
             ...    
874830    2023-11-22
874831    2023-11-22
874832    2023-11-22
874833    2023-11-22
874834    2023-11-22
Name: 일자, Length: 874654, dtype: object>
의안구분: ['건의안' '결의안' '규칙안' '동의안' '법률안' '예산안' '중요동의' '헌법개정' '의원징계' '승인안' '결산' '기타안'
 '윤리심사' '선출안' '의원자격심사']
단계: ['접수' '위원회 심사' '체계자구 심사' '공포' '본회의 심의' '정부이송' '예결위심사' '예비심사']
의안번호: ['020620' '020631' '030574' ... '2125569' '2125561' 'DD11506']
의안명: ['건의안' '결의안' '규칙안' '동의안' '법률안' '예산안' '중요동의' '헌법개정' '의원징계' '승인안' '결산' '기타안'
 '윤리심사' '선출안' '의원자격심사']
세부단계: ['제안' '회의' '회부' '공포' '정부이송' '처리' '상정' '본회의 심의' '의결']
소관위원회: ['-' '상공위원회' '법제사법위원회' '교통체신위원회' '보건사회위원회' '재정경제위원회' '본회의' '정치개혁특별위원회'
 '환경노동위원회' '건설교통위원회' '특별법기초위원회' '산업노동위원회' '내무치안위원회' '문교후생위원회' '외무국방위원회'
 '산업위원회' '문교사회위원회' '국방위원회' '내무위원회' '문교위원회' '사회보건위원회' '외무위원회' '농림위원회'
 '국회운영위원회' '각상임위원회' '예산결산위원회' '예산결산특별위원회' '부흥위원회' '징계자격위원회'
 '국가보안법개정안기

# 필요없는 Column 삭제

In [34]:
df = df.drop(columns=['대수'])

# 최근 의안건에 대해서만 필요하므로 2023년 이전 내용들은 삭제

In [61]:
# 일자가 0인 행 삭제
df = df[df['일자'] != '0']
# 966-09-16과 같은 오류값들을 Nat로 변환
df['일자'] = pd.to_datetime(df['일자'], errors='coerce')
# NaT 값을 가진 행 제거
df = df.dropna(subset=['일자']) 

df = df[df['일자'] >= pd.to_datetime('2023-01-01')]



In [62]:
# 일자 Column 확인
date_types = df['일자'].unique
print("일자: {}".format(date_types))

일자: <bound method Series.unique of 828120   2023-01-02
828121   2023-01-02
828122   2023-01-02
828123   2023-01-02
828124   2023-01-02
            ...    
874830   2023-11-22
874831   2023-11-22
874832   2023-11-22
874833   2023-11-22
874834   2023-11-22
Name: 일자, Length: 46704, dtype: datetime64[ns]>


In [70]:
# 의안 구분에 법률안 빼고 필요 없으므로 다 삭제
df = df[df['의안구분'] == '법률안']

# 세부 단계와 활동 상태는 같은 값을 가진 Column이므로 활동상태 Colum 삭제
df = df.drop(columns=['활동상태'])

In [72]:
# 각 Column들의 row 종류 확인
date_types = df['일자'].unique
print("일자: {}".format(date_types))
law_types = df['의안구분'].unique()
print("의안구분: {}".format(law_types))
turn_types = df['단계'].unique()
print("단계: {}".format(turn_types))
lawNum_types = df['의안번호'].unique()
print("의안번호: {}".format(lawNum_types))
lawName_types = df['의안명'].unique()
print("의안명: {}".format(law_types))
specific_types = df['세부단계'].unique()
print("세부단계: {}".format(specific_types))
insti_types = df['소관위원회'].unique()
print("소관위원회: {}".format(insti_types))


일자: <bound method Series.unique of 828120   2023-01-02
828121   2023-01-02
828122   2023-01-02
828123   2023-01-02
828124   2023-01-02
            ...    
874830   2023-11-22
874831   2023-11-22
874832   2023-11-22
874833   2023-11-22
874834   2023-11-22
Name: 일자, Length: 45393, dtype: datetime64[ns]>
의안구분: ['법률안']
단계: ['접수' '위원회 심사' '공포' '정부이송' '본회의 심의' '체계자구 심사']
의안번호: ['2119291' '2119294' '2119274' ... '2125561' 'DD11506' '2110624']
의안명: ['법률안']
세부단계: ['제안' '회부' '공포' '정부이송' '처리' '회의' '상정' '본회의 심의']
소관위원회: ['-' '보건복지위원회' '교육위원회' '국토교통위원회' '환경노동위원회' '행정안전위원회' '국회운영위원회' '법제사법위원회'
 '농림축산식품해양수산위원회' '과학기술정보방송통신위원회' '산업통상자원중소벤처기업위원회' '국방위원회' '기획재정위원회'
 '정무위원회' '여성가족위원회' '문화체육관광위원회' '본회의' '외교통일위원회' '정치개혁특별위원회' '정보위원회']


In [63]:
df.to_csv('날짜별_의정활동.csv', index=False)
